In [1]:
import pandas as pd
import numpy as np
import sys,requests
import time
import urllib.parse
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# IG rest API parameters

rest_api_key = "Hidden"
rest_identifier = "Hidden"
rest_password = "Hidden"

rest_url = "https://demo-api.ig.com/gateway/deal/session"

headers = {}
headers["Content-Type"] = "application/json; charset=UTF-8"
headers["Accept"] = "application/json; charset=UTF-8"
headers ["X-IG-API-KEY"] = rest_api_key
headers ["Version"] = "2"

request_json = {}
request_json["identifier"] = rest_identifier
request_json["password"] = rest_password

rest_response = requests.request("POST", rest_url, headers=headers, json=request_json)
if rest_response.status_code != 200:
    print("error", rest_response.status_code, rest_url, rest_response.text)
    sys.exit(0)
    
# collect params from login response

xst = rest_response.headers["X-SECURITY-TOKEN"]
cst = rest_response.headers["CST"]

In [3]:
# Get accounts list
headers["X-SECURITY-TOKEN"] = xst
headers["CST"] = cst
headers["Version"] = "1"
accounts = requests.request("GET", "https://demo-api.ig.com/gateway/deal/accounts", headers=headers,\
                            json=request_json)
if accounts.status_code != 200:
    print("error", accounts.status_code, accounts.text)

In [4]:
# Check account type is spreadbet, otherwise switch accounts
if rest_response.json()['accountType'] == 'SPREADBET':
    pass
else:
    account_json = {}
    account_json["accountId"] = "Hidden"
    swtich_response = requests.request("PUT", "https://demo-api.ig.com/gateway/deal/session/", headers=headers,\
                                  json=account_json)
    if swtich_response.status_code != 200:
        print("error", switch_reponse.status_code, switch_response.text)

In [9]:
# Place trade on US 500
headers["Version"] = "2"
order = {
    "epic": "IX.D.SPTRD.DAILY.IP",
    "expiry": None,
    "direction": "BUY",
    "size": "0.5",
    "orderType": None,
    "timeInForce": None,
    "level": None,
    "guaranteedStop": None,
    "stopLevel": None,
    "stopDistance": None,
    "trailingStop": None,
    "trailingStopIncrement": None,
    "forceOpen": None,
    "limitLevel": None,
    "limitDistance": None,
    "quoteId": None,
    "currencyCode": "GBP"
}

create_position = requests.request("POST", "https://demo-api.ig.com/gateway/deal/positions/otc", headers=headers,\
                                  json=order)

if create_position.status_code != 200:
    print("error", create_position.status_code, create_position.text)

error 400 {"errorCode":"validation.null-not-allowed.request.forceOpen"}


In [11]:
# Close position on US 500
headers["Version"] = "1"
close = {
    "dealId": None,
    "epic": "IX.D.SPTRD.DAILY.IP",
    "expiry": None,
    "direction": None,
    "size": "0.5",
    "level": None,
    "orderType": None,
    "timeInForce": None,
    "quoteId": None
}

close_position = requests.request("DELETE", "https://demo-api.ig.com/gateway/deal/positions/otc", headers=headers,\
                                 json=close)

if close_position.status_code != 200:
    print("error", close_position.status_code, close_position.text)

error 400 {"errorCode":"validation.null-not-allowed.request"}


In [14]:
# End session
headers["Version"] = "1"
log_out_details = {}

log_out = requests.request("DELETE", "https://demo-api.ig.com/gateway/session", headers=headers,\
                          json=log_out_details)
if log_out.status_code != 200:
    print("error", log_out.status_code, log_out.text)

In [7]:
### Use this code to download historical data ###
headers["Version"] = "2"

# S&P 500 = "IX.D.SPTRD.DAILY.IP"
# FTSE 100 = "IX.D.FTSE.DAILY.IP"

epic = "IX.D.FTSE.DAILY.IP"
resolution = "MINUTE_5"
start_date = "2020-02-04 02:00:00"
end_date = "2020-03-20 02:00:00"

data_url = f"https://demo-api.ig.com/gateway/deal/prices/{epic}/{resolution}/{start_date}/{end_date}"

historical_data = requests.request("GET", data_url, headers=headers)

if historical_data.status_code != 200:
    print("error", historical_data.status_code, historical_data.text)

In [8]:
# Convert historical data to pandas dataframe and export
t = time.localtime()
current_time = time.strftime("%M%H%d%m%y", t)
data = historical_data.json()['prices']

df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')

df.to_csv(f'Hidden.csv', encoding="UTF-8")